In [16]:
import numpy as np
import requests

In [12]:
url = 'http://cs.stanford.edu/people/karpathy/char-rnn/shakespear.txt'
f = requests.get(url)
data = f.text
chars = list(set(data)) # list of unique characters used in the data
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' %(data_size, vocab_size))
char_to_ix = {ch:i for i, ch in enumerate(chars)}
ix_to_char = {i:ch for i, ch in enumerate(chars)}

data has 99993 characters, 62 unique.


In [17]:
# hyperparameters
hidden_size = 100
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to out
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [28]:
def lossFunc(inputs, targets, hprev):
    """
    inputs, targets are both list of intergers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) #  hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y (http://cs231n.github.io/neural-networks-case-study/#grad)
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
    '''
    sample a sequence of integers from the model
    h is memory state, seed_ix is seed letter for first time step
    '''
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes= []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y)/np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) #3 memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size, 1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
    
    # sample from the model now and then
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ' '.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----'%(txt, ))
    
    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFunc(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print('iter %d, loss: %f'%(n, smooth_loss)) # print progress
    
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                 [dWxh, dWhh, dWhy, dbh, dby],
                                 [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

    p += seq_length # move data pointer
    n += 1 # iteration counter

----
 D q v Z W w D w c y : G c X c Z 
 ! i W l : w w   m . b v E v R k v J : S ? ! y : T V s S F B . c b Z ' F I U y k Z C G t t O ; B l m X p i U n F ; l ! D T : l - U M ! X k - ? a T ! d S W p ' M . z e x k S r c H z X A s A Y - g u . u S x u S T H X V l D p L w X Y s y A J B T 
 z J O X v W p O k B E I d l . O J   O S k . u j W m X k r k i o ; S g P z , Z ' Y C I q . E n t v ' r a h i ? D i 
 w x ' 
----
iter 0, loss: 103.178357
----
 S y S S A e S S S S S S h d S e S A A S A S S S S A S , S S h S t S S 
 u S n n , A s S t h S n S S S n S S d S S A i n o S A S n A S i S S S S S d S S 
 S S S S S S T S S g S n A 
 S b A S S S S S S a S S n t S S S S d A S S S A S S n S S S S S S S S S A S S u S A   n S S a S S S d S e m 
 S A S n S t S i S 
 S i S S S S 
 A S S a S t S d A r S S S d S S A S S S c S A 
 d S d S 
 A S S S S r S g 
----
iter 100, loss: 105.389982
----
 t s a w t   t s   u , k d t h a   a t h o u s o   s s t o o s t o t t p   n r o n   r o o t i e a t m s , t s   o   d 

----
 k e   D     Y l h   u     f a o o o F h d a m e   I y h , a i   t o   h   t b m o f b a i e h ' s e 
   t o t e r a     t o i a n r f u t m d   t s   v r   o r   e h   t u d o i e y   i s r s I   s e v s s   b N S   p o e   L o a   a   s a   e o i w r I b s   R f v   A i t   b r R m a e t   b i a   v h y m   e E s m Y l h d h   
 s l e   d l l   C 
   a   t r o , e o m , e I     m   h a b a g s 
 
----
iter 1900, loss: 87.670506
----
   r k h s S h e e F e e a b t i W b n O r A o l o e 
 i w a u h i   g e i   h P h   U   v i     c t t t e t   O   i r     U e A n h i r e e h e   l B s   A p t     r w r h e t h   i e s   r h e   e a   
 d t i r s t d s ! m l t   h n t p F t   
 e E t e a   S e m r y k   T o m t c r D e o b d a m e o 
 a s e l l I n r e t n l i b r n e e b L n F e e r y d I ;   r a a d e t o   e e h t t 
 a l s t 
----
iter 2000, loss: 87.454128
----
 n h s b   
 e e a f b s h b s S I i   e l   e l           d l   a e     J d t h g a   o e o P a e   e a s o u d r t 

----
 o n r d y t ,   a s I e m p s e :   h ,   .     b a o D h , h 
 m n     u u 
 ? e p o   B n t n t   c 
 a h 
   y   r a w . h s q s m e T   c t o s l p , a a d a h   r e a n s l , r e r o o c t e   T t t u g o P g o ? , n h a   r 
 m   
   n l   I v d     e c   n t a 
 
 e e t t   o   o h y e   o W   d   v     t a N w I m p ;   o i   s   I , i   d t t e a : m . a a h a n a n t u d h r   h   V s r 
----
iter 3800, loss: 83.821414
----
 T n e I e n o h   u t 
 e s a y n d a a   i   o c s W i : s t n ' 
   a f e a   e a i i N w a u n     h n   W n     M g W j n o   o A n W o o n p A   t s t n     e g i n p a b a   u l e 
 w k w h e i l a a d p t r , w   u 
 r n g r   , l   o o f s g   e e o u r a   o   ,   n 
 T o y 
   w I e r d e e i s u 
 u 
 o   : t   h u h o O H N a 
 
 n   n b a 
 v c o l r g e e s t a a s S r g y     h o   
----
iter 3900, loss: 83.680875
----
 o o   r h r l n   l n S a u c o C r   p h s a t o 
 n p o a r A i t i , i e s a   A , p   t a a e r O N t o     : r 

----
 h b c e d e   K 
 e o , h e ,   r O   o T u U i o w u i   s n r Y h   e e e p s   i h s e e s s I t y s   , h l t a i   c a n r e t a l h   u l o y s e , 
 e   B m   r d   I h t   e c o e h O i o u o h h y h s u d A   l r 
 s i t r w y d u   r n R s a h s   a b       s   o p t s f e h , l e   d s i t s c   e i t l s b k o e o e o y , t r m o n r   d e h o   u t b l h a n   h e l Y   i   
 y e h t 
----
iter 5700, loss: 82.858764
----
 i , w I , e b n o .   l n e o h   y t i s s     A o I d e   e , n o h ' z o n l   I o c   d c   t o e t t e d a h   w e I s e y   u d   i l d t , s r   t U I o b m d b v e s g O o l b 
 t a t a s e d e a d a ' o h r I   o h y u t l a I i e l ?   t e r r i i h t o h u ,   i . e     d g     f   e N a h i , n h o : s g s u e u s h ' t o i H o d 
 d h - 
 e g t b o t e s   s k t   e   E r   o     u   
----
iter 5800, loss: 82.776729
----
 f   f   l     r u   H T n u t l a i f u m a . v t s   f w e s t n ! o   o ! s   t w :   a l   f c T h r u e m i n e 

----
 i r 
 a l t   A p e i w s e H   t h i o t   o e o h e o t l i   e 
 o s t b d t 
   t   i     a h d r e n 
 M   b u h   n   s 
 e r e m E I   h r e i r y h i h f   o a   e r L i r o t t     e u c e e e N e n m , . I m a t 
 h   a d   t s c s   i r W s   s Q o p 
 A u s m s   , e r A ! f i h r s p p e e N h 
 i , T m h e a h   
   : , e V n     : A   U r k   h y t r r k O i m o i h     s t E s   r 
----
iter 7600, loss: 82.751418
----
 S A c d e , h i 
 r i l     
 t l i m o m S a r , a o U e o E f l l c s t . M t T   f   n s m o n s a   o t S t   c g a c e i O o A m ' o t h H I I M t r l n 
 n h W w e f t r g p     y h   K c h s , t l r e   , t f a 
 d t s n c Y D g i z w t w e N 
 e e o r u 
 , i e 
 i o o v e s s s o   r c 
 y t u d h n   c c r m a   f o o r c s c S i n u u 
 e r U : R w   Y o E f   y e   a , i e x ? n L a n 
----
iter 7700, loss: 82.848118
----
 m Y     u d e r s   p 
 h n t a s 
   d   w n I , h h I e o   a t d m t a t   m s M i t s n n n d k e w f h t t A   

...

----
 t e g y a s h l   
 a a L p e a n f u e     e n r a F e 
 O t l e s R   u e h ' r d a c   d b a i e e h r I s a n t e s e     m 
     e l S   ! n c E a r o t   F h a d   t B 
 r w n t - i h   u   m s     o w f     r p   y r y u i t p N d F e t m I u s v r m   h H e a : g 
 k 
 s h i 
   N i b e w d l e h o l   g y i l o s r a   e h   n n     a : h     e u b e y s ; d   w w         s o h   o a   a 
----
iter 150100, loss: 81.951308
----
 r h   T   R s o u a a A o r i R 
 p e e g o   :       i d a n o   f o y w s   B s e C y e   n f e 
   p w h N t   i t T 
 C b l s r w i n f   a n   t v s   l c n : i m h r m i I g a o a h r l r   d H i , a w A s r f     m   a c t d e h n   m t     m l o u d Y i e l o u N O t d h n   s o   m n a n   e w s r e h   s   w   a 
 b e y g e o Q   n t H t h h m t b t r 
 p   i s n u E i , r F   A , e   w 
----
iter 150200, loss: 81.894684
----
   n t   n t e 
 y t     A h s m h ' v s e a a l B g e   r   : m a   
 l u h y i e g a r i r r     e ,   k   h   

----
 r o   I a d r d   y n l u A f e a e F , l   n t v e I g t d i   d m a h e   U : r K r r a e b n   r e h   i p h e u s N o i T I o r t b w e n e o d s u a   b a n e t s   s t a e W   h n n r o   h d   h   m     e m h   e : s i a   O d e b   H   I       t h a l     o s n s s d o u t e u v n b a   b   r p   n e   r a   l f t e 
 l l n i   i A o   h   I s r t i i   n u 
 n ' o   t   
 o f e h o s e t 
----
iter 152000, loss: 81.883522
----
 f N k a t r o o s b t n n e e d m t   H l o a   i e w h h   y a h e s U o     i r t g o p r h   t ,     n 
 e r   a s v ' r   o ,     n o w e o I   m O     a y u A c e t   r e , s   r   i K h ' t a h i e g c h a f i t G     e   y   r e d o h n l O n   I i L 
 V   w     ,   a     m h s d   ; o f   f 
 I n e e     . , b 
 a , h d i r ' n a l R   a o a   s a e   e . e   h g s s A , N         r n n b 
----
iter 152100, loss: 81.934586
----
     m   s e i   t A b h s i p   a s 
 m a e a t o s d w n o s e k   l e S   v l l o l M o e s o   
 n e   e , h 

----
 e r r l n n b t f o   u s l     U :     d a e h l t     E e s   o n   , u e e u   g h u i , o d P t l e h 
 w s g 
   e v p o r h i l   o f e m O h e o d o :     l e e r V 
 i   e n   t t o h d i f h e   F s t r   Y o n W e a u o e   s c s e e B .   t O e o ' e   m e e r t h ! E e u e   i l a     E h h   i   ' I a a o e l   e . y ? v   r w e e     r a o     T   f e n o i a I t T e r o   N d e i   
----
iter 153900, loss: 81.873238
----
 t I o m , , e l i e d e e i n 
 e     s n L d s w a r e e w 
 r u   s t o u a s i   
 
 N n l u u a h   p s l b   i h t r a   t i y A I o       n w F i e u s   e , e     t o         o e   w   
 m h     t e n i o   a y a o t 
 t f 
 t a   e   f e h n   , o e S u .   e M N h i o s l d h B l e t   , s o u ;       e   e   r s   l   
 e e t 
 s a 
   k s r A N h u r L u   t 
 o u o A   e w d   s D t y 
----
iter 154000, loss: 82.080012
----
 w v t i u s l   C   m l t r t   e b s ' d s   u e a h a t p     u e b e t       u a t f h y s i k d m i 
 h 
 d 

----
 i ' s o t t e h t n   o   r a   s s e     d   a o   i t r 
 i W g c s s m h 
   e a l m     e s L s s r w n w   e d e t h s e     i t   i d D u .   L n u I m 
 C h u h   e i h     u       s a u i A g : 
 s 
 h f a s s i t s t   m i a -   t l b p u   a f n h e l   k   t i h   u e   o h s a u   r t l e   o 
 y e e   a i u u y e d   v A m b ' h A . s h a b m e e '     s a h s e i C s h U t n h   l r 
----
iter 155800, loss: 81.887904
----
   r   u s   l t e   a a . i a o r p o c r : B d : i m F a s   e 
   . M i e U o h   a a d y O 
 a ! d o t n o r a y s c g o o   i e e o T :   s d ! 
 e s e s   b A 
   a g t   d P u n B s e d o c u o o a f w 
 d 
 d A a   O u o g l l k d i o   n o l ,   A   r a u   m h U v   h   m   e S 
 s i i b O h   e a     u i a r e a 
 i e   k i o t n e o 
 s l m t a n r e u d     g p u 
 d   
 e 
 d e o   i 
----
iter 155900, loss: 82.012298
----
 c h h y l i C   a g e f d d   i e   o t L t m 
 i   h t d . d v . t e h d   n n s i g o e w g a i n t 
 u i s 
 

KeyboardInterrupt: 

In [47]:
# Gradient checking
from random import uniform
def gradCheck(inputs, target, hyprev):
    global Wxh, Whh, Why, bh, by
    num_checks, delta = 10, 1e-5
    _, dWxh, dWhh, dWhy, dbh, dby, _ = lossFunc(inputs, targets, hprev)
    for param, dparam, name in zip([Wxh, Whh, Why, bh, by], [dWxh, dWhh, dWhy, dbh, dby], ['Wxh', 'Whh', 'Why', 'bh', 'by']):
        s0 = dparam.shape
        s1 = param.shape
        assert s0 == s1, ('Error dims don\'t match: %s and %s.'%('s0', 's1'))
        print(name)
        for i in range(num_checks):
            ri = int(uniform(0,param.size))
            # evaluate cost at [x + delta] and [x - delta]
            old_val = param.flat[ri]
            param.flat[ri] = old_val + delta
            cg0, _, _, _, _, _, _ = lossFunc(inputs, targets, hprev)
            param.flat[ri] = old_val - delta
            cg1, _, _, _, _, _, _ = lossFunc(inputs, targets, hprev)
            param.flat[ri] = old_val # reset old values for this parameter
            # fetch both numerical and analytic gradient
            grad_analytic = dparam.flat[ri]
            grad_numerical = (cg0 - cg1)/(2*delta)
            rel_error = abs(grad_analytic - grad_numerical)/abs(grad_numerical + grad_analytic)
            print('%f, %f => %e'%(grad_numerical, grad_analytic, rel_error))
            #rel_error should be on order of 1e-7 or less

In [48]:
gradCheck(inputs, targets, hprev)

Wxh
0.000000, 0.000000 => nan
0.000000, 0.000000 => nan
0.000000, 0.000000 => nan
0.000000, 0.000000 => nan
0.000000, 0.000000 => nan
0.000000, 0.000000 => nan
0.000000, 0.000000 => nan
0.000000, 0.000000 => nan
0.000000, 0.000000 => nan
0.000000, 0.000000 => nan
Whh
0.000000, -0.000000 => 1.000000e+00

C:\Users\user\Miniconda3\lib\site-packages\ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in double_scalars



0.000000, -0.000000 => 1.016031e+00
-0.000000, -0.000000 => 9.872062e-01
0.000000, -0.000000 => 1.045452e+00
-0.000000, -0.000000 => 9.800938e-01
0.000000, -0.000000 => 1.000000e+00
-0.000000, -0.000000 => 9.706454e-01
-0.000000, -0.000000 => 9.701003e-01
-0.000000, -0.000000 => 9.864600e-01
0.000000, -0.000000 => 1.000000e+00
Why
-0.054197, -0.054197 => 4.399912e-09
0.663540, 0.663540 => 4.670456e-10
-0.093671, -0.093671 => 2.250403e-09
-0.028229, -0.028229 => 4.363076e-09
-0.008406, -0.008406 => 2.225238e-08
-0.400686, -0.400686 => 1.803495e-10
-0.151911, -0.151911 => 5.595551e-10
0.496948, 0.496948 => 3.316014e-10
-0.164594, -0.164594 => 2.262374e-09
-0.050483, -0.050483 => 4.161673e-09
bh
0.000000, 0.000000 => 1.000000e+00
-0.000000, 0.000000 => 1.029025e+00
0.000000, 0.000000 => 9.641158e-01
-0.000000, 0.000000 => 1.007103e+00
0.000000, 0.000000 => 9.786772e-01
-0.000000, 0.000000 => 1.016031e+00
0.000000, 0.000000 => 9.786772e-01
0.000000, 0.000000 => 9.931306e-01
-0.000000, 0.0